In [4]:
import os
import sys
from pathlib import Path

from loguru import logger
from reload_recursive import reload_recursive
from typing import Callable

from nipype.interfaces import fsl

import monai_training
import mri_data
import sys

In [5]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data import file_manager as fm
from mri_data.file_manager import Scan, DataSet
from mri_data import utils
from monai_training.preprocess import DataSetProcesser

In [6]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [7]:
drive_root = fm.get_drive_root()
projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
inference_root = drive_root / "3Tpioneer_bids_predictions"
dataroot = drive_root / "3Tpioneer_bids"

workspace_root = "/home/srs-9/Projects/ms_mri/itksnap_workspaces"
dataset_name = "choroid_pineal_pituitary_labels"

save_dir = os.path.join(workspace_root, dataset_name)

In [8]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [9]:
subjects = [
    1010,
    1011,
    1019,
    1033,
    1065,
    1080,
    1109,
    1119,
    1152,
    1163,
    1188,
    1191,
    1234,
    1259,
    1265,
    1272,
    1280,
    1293,
    1321,
    1355,
    1437,
    1486,
    1498,
    1518,
    1540,
    1547,
    1548,
    2081,
    2083,
    2097,
    2126,
    2132,
    2144,
    2146,
    2164,
    2187,
]

In [10]:
def has_subject(scan, subjects: list[int]) -> bool:
    if int(scan.subid) in subjects:
        return True
    else:
        return False

In [11]:
dataset_proc = DataSetProcesser.new_dataset(dataroot, fm.scan_3Tpioneer_bids, filters=[fm.filter_first_ses])
dataset_proc.filter([has_subject], [(subjects,)])
dataset = dataset_proc.dataset

FileNotFoundError: [Errno 2] No such file or directory: '/media/smbshare/3Tpioneer_bids'

In [ ]:
dataset_proc.prepare_labels(["choroid_t1_flair", "pineal", "pituitary"], ["CH", "SRS", "DT", "ED"])
dataset_proc.prepare_images(["flair", "t1"])

2024-11-05 20:33:43.858 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
  0%|          | 0/36 [00:00<?, ?it/s]2024-11-05 20:33:43.865 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1547, sesid=20220321)
2024-11-05 20:33:43.904 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms1547/ses-20220321/choroid_t1_flair-ED.pineal-SRS.pituitary-CH.nii.gz
2024-11-05 20:33:43.906 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1033, sesid=20171117)
2024-11-05 20:33:43.928 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms1033/ses-20171117/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz
2024-11-05 20:33:43.930 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1259, sesid=20200803)
2024-11-05 20:33:43.941 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms1259/ses-20200803/choroid_t1_f

In [14]:
for scan in dataset_proc.dataset:
    images = [str(scan.root / "flair.nii.gz"), str(scan.root / "t1.nii.gz")]
    labels = [str(scan.label_path)]
    cmd = utils.open_itksnap_workspace_cmd(images, labels)
    print(cmd)

itksnap -g H:/3Tpioneer_bids/sub-ms1547/ses-20220321/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1547/ses-20220321/t1.nii.gz -s H:/3Tpioneer_bids/sub-ms1547/ses-20220321/choroid_t1_flair-ED.pineal-SRS.pituitary-CH.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1033/ses-20171117/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1033/ses-20171117/t1.nii.gz -s H:/3Tpioneer_bids/sub-ms1033/ses-20171117/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1259/ses-20200803/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1259/ses-20200803/t1.nii.gz -s H:/3Tpioneer_bids/sub-ms1259/ses-20200803/choroid_t1_flair-ED.pineal-SRS.pituitary-CH.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms2081/ses-20170204/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms2081/ses-20170204/t1.nii.gz -s H:/3Tpioneer_bids/sub-ms2081/ses-20170204/choroid_t1_flair-ED.pineal-SRS.pituitary-CH.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1188/ses-20200720/t1.nii.gz -s H:/3Tpion

In [31]:
labels = ["thoo.nii.gz"]
" -s ".join(labels).split(" ")

['thoo.nii.gz']

In [22]:
dataset = dataset_proc.dataset
subjs = set([int(scan.subid) for scan in dataset])

set(subjects) - subjs

{1152, 2097, 2126, 2132, 2146, 2164, 2187}

In [36]:
subs = [1010, 1011, 1019, 1033, 1065, 1080, 1109, 1119, 1152, 1163, 1188, 1191, 1234, 1259, 1265, 1272, 1280, 1293, 1321, 1355, 1437, 1486, 1498, 1518, 1518, 1540, 1547, 1548, 2081, 2083, 2097, 2126, 2132, 2144, 2146, 2164, 2187]

len(subs)

37

In [23]:
result.outputs.out_stat[1]

1003.0